In [6]:
from classes import *
import matplotlib
game_settings = GameSettings(starting_capital = 10,
                        no_emp_cards_in_pool = 8,
                        no_emp_cards_per_attr = 16,
                        no_bud_cards_in_pool = 6,
                        no_bud_cards_per_attr = 16,
                        desks_per_office = 2,
                        production_per_factory = 2,
                        cost_per_office = 2,
                        cost_per_factory = 2,
                        engineering_to_unit_cost = {0:6,1:5,2:4,3:3,4:2},
                        marketing_to_brand = {0:1,1:2,2:3,3:4,4:5},
                        finance_to_max_gross = {0:2,1:6,2:12,3:18,4:24},
                        operations_to_max_buildings = {0:1,1:2,2:3,3:4,4:5},
                        price_to_demand = {3:5,4:4,5:3,6:2,7:1}
                       )

In [7]:
game = Game(game_settings, no_players = 1)
print(game.employee_deck.cards[0])
print(game.employee_deck.cards[1])
print(game.employee_deck.cards[2])
print(game.employee_deck.cards[3])
print(game.building_deck.cards[0])
print(game.building_deck.cards[1])
print(game.building_deck.cards[2])

employee, name: Sam Roberts, department: operations, cost: 1, operations: 1, finance: 0, engineering: 0, marketing: 0
employee, name: John Jones, department: finance, cost: 1, operations: 0, finance: 1, engineering: 0, marketing: 0
employee, name: Sam Sinclair, department: engineering, cost: 1, operations: 0, finance: 0, engineering: 1, marketing: 0
employee, name: Jack Smith, department: marketing, cost: 1, operations: 0, finance: 0, engineering: 0, marketing: 1
building, type: factory, cost: 1.3333333333333333, production: 2, storage: 0, desks: 0
building, type: warehouse, cost: 2.0, production: 0, storage: 3, desks: 0
building, type: office, cost: 1.3333333333333333, production: 0, storage: 0, desks: 2


In [8]:
res = game.players[0].generate_game_states(game_settings, game.companies[0], iterations = 200, debug = 1)
res[:10]

,operations,finance,engineering,marketing,offices,factories,unit cost,price,net
13,0,0,0,0,0,1,6,7,0.666667
8,0,0,0,0,0,0,6,7,0.000000
158,0,0,0,0,1,0,6,7,-1.333333
24,1,0,0,0,1,1,6,7,-1.666667
36,0,0,0,1,1,0,6,7,-2.333333
92,0,0,1,0,1,0,5,7,-2.333333
34,1,0,0,0,1,0,6,7,-2.333333
14,2,0,0,0,1,1,6,7,-2.666667
37,1,1,0,0,1,1,6,7,-2.666667
98,1,0,1,0,1,1,5,7,-2.666667


In [5]:
game_state = (0,0,0,0,0,1,7)
net = game.companies[0].analyse_game_state(game_state, debug=1)

A, employee attributes: operations: 0, engineering: 0, finance: 0, marketing: 0
B, building attributes: production: 3, storage: 0, desks: 0, max buildings: 1
C, unit cost: 6, brand demand: 1, price demand: 1, demand strength: 2, actual demand: 2
D, potential gross: 2, max gross: 2, actual gross: 2, total cost: 2.0, net: 0.0


In [9]:
game_state = (0,0,0,0,0,1,7)
net = game.companies[0].analyse_game_state(game_state, debug=1)

A, employee attributes: operations: 0, engineering: 0, finance: 0, marketing: 0
B, building attributes: production: 2, storage: 0, desks: 0, max buildings: 1
C, unit cost: 6, brand demand: 1, price demand: 1, demand strength: 2, actual demand: 2
D, potential gross: 2, max gross: 2, actual gross: 2, total cost: 1.3333333333333333, net: 0.6666666666666667
